<a href="https://colab.research.google.com/github/dalasjoe-1/dfl/blob/master/GetLotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ALL Together

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os import path
from google.colab import drive

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-rc1

main_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin" # 마지막 회차를 얻기 위한 주소
basic_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin&drwNo=" # 임의의 회차를 얻기 위한 주소

# 마지막 회차 정보를 가져옴
def GetLast(): 
    resp = requests.get(main_url)
    soup = BeautifulSoup(resp.text, "lxml")
    result = str(soup.find("meta", {"id" : "desc", "name" : "description"})['content'])
    s_idx = result.find(" ")
    e_idx = result.find("회")
    return int(result[s_idx + 1 : e_idx])

# 지정된 파일에 지정된 범위의 회차 정보를 기록함
def Crawler(s_count, e_count, fp):
    for i in range(s_count , e_count + 1):
        crawler_url = basic_url + str(i)
        resp = requests.get(crawler_url)
        soup = BeautifulSoup(resp.text, "html.parser")

        text = soup.text

        s_idx = text.find(" 당첨결과")
        s_idx = text.find("당첨번호", s_idx) + 4
        e_idx = text.find("보너스", s_idx)
        numbers = text[s_idx:e_idx].strip().split()

        s_idx = e_idx + 3
        e_idx = s_idx + 3
        bonus = text[s_idx:e_idx].strip()

        s_idx = text.find("1등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money1 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("2등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money2 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("3등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money3 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("4등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money4 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("5등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money5 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        line = str(i) + ',' + numbers[0] + ',' + numbers[1] + ',' + numbers[2] + ',' + numbers[3] + ',' + numbers[4] + ',' + numbers[5] + ',' + bonus + ',' + money1 + ',' + money2 + ',' + money3 + ',' + money4 + ',' + money5
        print(line)
        line += '\n'
        fp.write(line)

last = GetLast() # 마지막 회차를 가져옴

lotto_dir_name='lotto'
drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
if not path.exists(lotto_base_dir):
  print('Check your google drive directory. See you file explorer')


print ("Last=",last)

# 완전히 다시 쓰기
#fp = open(path.join(lotto_base_dir, "lotto_data.txt"), 'w')

# 기존 데이타에 추가하기
fp = open(path.join(lotto_base_dir, "lotto_data.txt"), 'a')

#Crawler(last-1, last, fp) # 처음부터 마지막 회차까지 저장
#fp.close()



# 최대 100번 에포크까지 수행

model = tf.keras.models.load_model('./drive/MyDrive/my_model.h5')
model.summary()

for epoch in range(50):

    model.reset_states() # 중요! 매 에포크마다 1회부터 다시 훈련하므로 상태 초기화 필요

    batch_train_loss = []
    batch_train_acc = []

    for i in range(len(x_samples)):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss)))  


# Trainning 끝나면 무조건 모델에 저장
model.save('my_model.h5')
!cp /content/my_model.h5 /content/drive/My\ Drive/


from ftplib import FTP

ftp = FTP('112.175.184.78')
ftp.login('dalasjoe', 'Dalasjoe75!')

# ftp.cwd('html') # "test"디렉터리로 이동
# ftp.retrlines('LIST') # 디렉터리의 내용을 목록화
# #ftp.retrbinary('RETR README', open('README', 'wb').write) # README 파일 저장
# ftp.quit()


ftp.cwd('html')  # 업로드할 FTP 폴더로 이동
myfile = open(path.join(lotto_base_dir, "predict2.txt"),'rb')  # 로컬 파일 열기
ftp.storbinary('STOR ' + 'predict2.txt', myfile )  # 파일을 FTP로 업로드
myfile.close()  # 파일 닫기

print ("File Saved")

# 저장된 데이터 확인

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os import path
from google.colab import drive

otto_dir_name='lotto'
drive.mount('/content/gdrive',force_remount=True)

if not path.exists(lotto_base_dir):
  print('Check your google drive directory. See you file explorer')

lotto_base_dir=path.join('./gdrive/My Drive/', '')
with open(path.join(lotto_base_dir, "lotto_data.txt"), "r") as f:
  print("11111")
  line = f.read()
  print(">>>" + line)


# **데이터 수집**  - Crawling

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os import path
from google.colab import drive

main_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin" # 마지막 회차를 얻기 위한 주소
basic_url = "https://www.dhlottery.co.kr/gameResult.do?method=byWin&drwNo=" # 임의의 회차를 얻기 위한 주소

# 마지막 회차 정보를 가져옴
def GetLast(): 
    resp = requests.get(main_url)
    soup = BeautifulSoup(resp.text, "lxml")
    result = str(soup.find("meta", {"id" : "desc", "name" : "description"})['content'])
    s_idx = result.find(" ")
    e_idx = result.find("회")
    return int(result[s_idx + 1 : e_idx])

# 지정된 파일에 지정된 범위의 회차 정보를 기록함
def Crawler(s_count, e_count, fp):
    for i in range(s_count , e_count + 1):
        crawler_url = basic_url + str(i)
        resp = requests.get(crawler_url)
        soup = BeautifulSoup(resp.text, "html.parser")

        text = soup.text

        s_idx = text.find(" 당첨결과")
        s_idx = text.find("당첨번호", s_idx) + 4
        e_idx = text.find("보너스", s_idx)
        numbers = text[s_idx:e_idx].strip().split()

        s_idx = e_idx + 3
        e_idx = s_idx + 3
        bonus = text[s_idx:e_idx].strip()

        s_idx = text.find("1등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money1 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("2등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money2 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("3등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money3 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("4등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money4 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        s_idx = text.find("5등", e_idx) + 2
        e_idx = text.find("원", s_idx) + 1
        e_idx = text.find("원", e_idx)
        money5 = text[s_idx:e_idx].strip().replace(',','').split()[2]

        line = str(i) + ',' + numbers[0] + ',' + numbers[1] + ',' + numbers[2] + ',' + numbers[3] + ',' + numbers[4] + ',' + numbers[5] + ',' + bonus + ',' + money1 + ',' + money2 + ',' + money3 + ',' + money4 + ',' + money5
        print(line)
        line += '\n'
        fp.write(line)

last = GetLast() # 마지막 회차를 가져옴

lotto_dir_name='lotto'
drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
if not path.exists(lotto_base_dir):
  print('Check your google drive directory. See you file explorer')

#with open(path.join(lotto_base_dir, "lotto_data.txt"), "r") as f:
#  print("11111")
#   line = f.read()
#   print(">>>" + line)

print ("Last=",last)

# 완전히 다시 쓰기
#fp = open(path.join(lotto_base_dir, "lotto_data.txt"), 'w')

# 기존 데이타에 추가하기
fp = open(path.join(lotto_base_dir, "lotto_data.txt"), 'a')

Crawler(1, last, fp) # 처음부터 마지막 회차까지 저장
fp.close()


## 데이터 로딩

In [5]:
from os import path
from google.colab import drive
import numpy as np

drive.mount('/content/drive')

rows = np.loadtxt("./drive/MyDrive/lotto_data.txt", delimiter=",")
row_count = len(rows)

print("Data Loaded : row count: " + str(row_count))

Mounted at /content/drive
Data Loaded : row count: 965


# **TensorFlow 설치 및 모델 초기화**

In [36]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

from __future__ import absolute_import, division, print_function, unicode_literals
!pip install tensorflow-gpu==2.0.0-rc1

# 당첨번호를 원핫인코딩벡터(ohbin)으로 변환
def numbers2ohbin(numbers):

    ohbin = np.zeros(45) #45개의 빈 칸을 만듬

    for i in range(6): #여섯개의 당첨번호에 대해서 반복함
        ohbin[int(numbers[i])-1] = 1 #로또번호가 1부터 시작하지만 벡터의 인덱스 시작은 0부터 시작하므로 1을 뺌
    
    return ohbin

# 원핫인코딩벡터(ohbin)를 번호로 변환
def ohbin2numbers(ohbin):

    numbers = []
    
    for i in range(len(ohbin)):
        if ohbin[i] == 1.0: # 1.0으로 설정되어 있으면 해당 번호를 반환값에 추가한다.
            numbers.append(i+1)
    
    return numbers


numbers = rows[:, 1:7]
ohbins = list(map(numbers2ohbin, numbers))

x_samples = ohbins[0:row_count-1]
y_samples = ohbins[1:row_count]

#원핫인코딩으로 표시
print("ohbins")
print("X[0]: " + str(x_samples[0]))
print("Y[0]: " + str(y_samples[0]))

#번호로 표시
print("numbers")
print("X[0]: " + str(ohbin2numbers(x_samples[0])))
print("Y[0]: " + str(ohbin2numbers(y_samples[0])))

# 데이터셋 구성
train_idx = (0, 700) # 훈련셋 
val_idx = (700, 800) # 검증셋
test_idx = (800, len(x_samples)) #시험셋

print("train: {0}, val: {1}, test: {2}".format(train_idx, val_idx, test_idx))

# 모델을 정의합니다. - LSTM
#타입스텝은 1인 대신, 상태유지(stateful 옵션을 True)으로 설정했습니다.
#45개의 벡터로 출력합니다.
#각각의 벡터는 0.0과 1.0사이의 실수값으로 나옵니다. 각 벡터가 독립적으로 모두 1.0이 나오거나 0.0이 나올 수 있는 멀티레이블 문제입니다.
#멀티레이블 문제라 출력층의 활성화함수가 softmax가 아닌 sigmoid로 설정하였습니다.
model = keras.Sequential([
    keras.layers.LSTM(128, batch_input_shape=(1, 1, 45), return_sequences=False, stateful=True),
    keras.layers.Dense(45, activation='sigmoid')
])

# 모델을 컴파일합니다.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 매 에포크마다 훈련과 검증의 손실 및 정확도를 기록하기 위한 변수
train_loss = []
train_acc = []
val_loss = []
val_acc = []


# 88회부터 지금까지 1등부터 5등까지 상금의 평균낸다.
mean_prize = [  np.mean(rows[87:, 8]),
            np.mean(rows[87:, 9]),
            np.mean(rows[87:, 10]),
            np.mean(rows[87:, 11]),
            np.mean(rows[87:, 12])]

print(mean_prize)   

train_total_reward = []
train_total_grade = np.zeros(6, dtype=int)

val_total_reward = []
val_total_grade = np.zeros(6, dtype=int)

test_total_reward = []
test_total_grade = np.zeros(6, dtype=int)

model.reset_states()

# print('[No. ] 1st 2nd 3rd 4th 5th 6th Rewards')

# for i in range(len(x_samples)):
#     xs = x_samples[i].reshape(1, 1, 45)
#     ys_pred = model.predict_on_batch(xs) # 모델의 출력값을 얻음
    
#     sum_reward = 0
#     sum_grade = np.zeros(6, dtype=int) # 6등까지 변수

#     for n in range(10): # 10판 수행
#         numbers = gen_numbers_from_probability(ys_pred[0])
        
#         #i회차 입력 후 나온 출력을 i+1회차와 비교함
#         grade, reward = calc_reward(rows[i+1,1:7], rows[i+1,7], numbers) 

#         sum_reward += reward
#         sum_grade[grade] += 1

#         if i >= train_idx[0] and i < train_idx[1]:
#             train_total_grade[grade] += 1
#         elif i >= val_idx[0] and i < val_idx[1]:
#             val_total_grade[grade] += 1
#         elif i >= test_idx[0] and i < test_idx[1]:
#             val_total_grade[grade] += 1
    
#     if i >= train_idx[0] and i < train_idx[1]:
#         train_total_reward.append(sum_reward)
#     elif i >= val_idx[0] and i < val_idx[1]:
#         val_total_reward.append(sum_reward)
#     elif i >= test_idx[0] and i < test_idx[1]:
#         test_total_reward.append(sum_reward)
                        
#     print('[{0:4d}] {1:3d} {2:3d} {3:3d} {4:3d} {5:3d} {6:3d} {7:15,d}'.format(i+1, sum_grade[0], sum_grade[1], sum_grade[2], sum_grade[3], sum_grade[4], sum_grade[5], int(sum_reward)))

# print('Total') 
# print('==========')    
# print('Train {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(train_total_grade[0], train_total_grade[1], train_total_grade[2], train_total_grade[3], train_total_grade[4], train_total_grade[5], int(sum(train_total_reward))))
# print('Val   {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(val_total_grade[0], val_total_grade[1], val_total_grade[2], val_total_grade[3], val_total_grade[4], val_total_grade[5], int(sum(val_total_reward))))
# print('Test  {0:5d} {1:5d} {2:5d} {3:5d} {4:5d} {5:5d} {6:15,d}'.format(test_total_grade[0], test_total_grade[1], test_total_grade[2], test_total_grade[3], test_total_grade[4], test_total_grade[5], int(sum(test_total_reward))))
# print('==========')   

ohbins
X[0]: [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
Y[0]: [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
numbers
X[0]: [10, 23, 29, 33, 37, 40]
Y[0]: [9, 13, 21, 25, 32, 42]
train: (0, 700), val: (700, 800), test: (800, 964)
[2432219832.8667426, 57182024.93166287, 1451067.9487471527, 52814.325740318905, 5000.0]


In [37]:
model = tf.keras.models.load_model('./drive/MyDrive/my_model.h5')
model.summary()

OSError: ignored

# **Trainning**

In [35]:
# 최대 100번 에포크까지 수행

# function ClickConnect(){
#     console.log("코랩 연결 끊김 방지"); 
#     document.querySelector("colab-toolbar-button#connect").click() 
# }
# setInterval(ClickConnect, 60 * 1000)

model = tf.keras.models.load_model('./drive/MyDrive/my_model.h5')
model.summary()

for epoch in range(50):

    model.reset_states() # 중요! 매 에포크마다 1회부터 다시 훈련하므로 상태 초기화 필요

    batch_train_loss = []
    batch_train_acc = []

    for i in range(len(x_samples)):
        
        xs = x_samples[i].reshape(1, 1, 45)
        ys = y_samples[i].reshape(1, 45)
        
        loss, acc = model.train_on_batch(xs, ys) #배치만큼 모델에 학습시킴

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss)))  


# Trainning 끝나면 무조건 모델에 저장
model.save('my_model.h5')
!cp /content/my_model.h5 /content/drive/My\ Drive/

# 마지막 회차까지 학습한 모델로 다음 회차 추론

 # 등수와 상금을 반환함
# 순위에 오르지 못한 경우에는 등수가 0으로 반환함
def calc_reward(true_numbers, true_bonus, pred_numbers):

    count = 0

    for ps in pred_numbers:
        if ps in true_numbers:
            count += 1

    if count == 6:
        return 0, mean_prize[0]
    elif count == 5 and true_bonus in pred_numbers:
        return 1, mean_prize[1]
    elif count == 5:
        return 2, mean_prize[2]
    elif count == 4:
        return 3, mean_prize[3]
    elif count == 3:
        return 4, mean_prize[4]

    return 5, 0

def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

   
    selected_balls.sort()

    return selected_balls

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []

# 50 개 뽑기
# for n in range(100):
#     numbers = gen_numbers_from_probability(ys_pred[0])  
#     print('{0} : {1}'.format(n, numbers))    
#     list_numbers.append(numbers)  


drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
with open(path.join(lotto_base_dir, "predict.txt"), "w") as f:
  print ("predict.txt")

  for n in range(50):
    numbers = gen_numbers_from_probability(ys_pred[0])  
    print('{0} : {1}'.format(n, numbers))    
    list_numbers.append(numbers)  
    line_str=','.join(str(e) for e in numbers)
    line_str += '\n'
    print(line_str)
    f.write(line_str)
f.close()

OSError: ignored

# **Prediction**

In [ ]:
# 마지막 회차까지 학습한 모델로 다음 회차 추론

 # 등수와 상금을 반환함
# 순위에 오르지 못한 경우에는 등수가 0으로 반환함
def calc_reward(true_numbers, true_bonus, pred_numbers):

    count = 0

    for ps in pred_numbers:
        if ps in true_numbers:
            count += 1

    if count == 6:
        return 0, mean_prize[0]
    elif count == 5 and true_bonus in pred_numbers:
        return 1, mean_prize[1]
    elif count == 5:
        return 2, mean_prize[2]
    elif count == 4:
        return 3, mean_prize[3]
    elif count == 3:
        return 4, mean_prize[4]

    return 5, 0

def gen_numbers_from_probability(nums_prob):

    ball_box = []

    for n in range(45):
        ball_count = int(nums_prob[n] * 100 + 1)
        ball = np.full((ball_count), n+1) #1부터 시작
        ball_box += list(ball)

    selected_balls = []

    while True:
        
        if len(selected_balls) == 6:
            break
        
        ball_index = np.random.randint(len(ball_box), size=1)[0]
        ball = ball_box[ball_index]

        if ball not in selected_balls:
            selected_balls.append(ball)

   
    selected_balls.sort()

    return selected_balls

print('receive numbers')

xs = x_samples[-1].reshape(1, 1, 45)

ys_pred = model.predict_on_batch(xs)

list_numbers = []

# 50 개 뽑기
# for n in range(100):
#     numbers = gen_numbers_from_probability(ys_pred[0])  
#     print('{0} : {1}'.format(n, numbers))    
#     list_numbers.append(numbers)  


drive.mount('/content/gdrive',force_remount=True)
lotto_base_dir=path.join('./gdrive/My Drive/', '')
with open(path.join(lotto_base_dir, "predict.txt"), "w") as f:
  print ("predict.txt")

  for n in range(50):
    numbers = gen_numbers_from_probability(ys_pred[0])  
    print('{0} : {1}'.format(n, numbers))    
    list_numbers.append(numbers)  
    line_str=','.join(str(e) for e in numbers)
    line_str += '\n'
    print(line_str)
    f.write(line_str)
f.close()

# **Save Model**

In [ ]:
model.save('my_model.h5')

# **Load Model**

In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from os import path
from google.colab import drive

new_model = tf.keras.models.load_model('./drive/MyDrive/my_model.h5')
new_model.summary()

OSError: ignored

In [ ]:
!cp /content/my_model.h5 /content/drive/My\ Drive/

cp: failed to access '/content/drive/My Drive/': Transport endpoint is not connected


# FTP 전송

In [31]:
from ftplib import FTP

ftp = FTP('112.175.184.78')
ftp.login('dalasjoe', 'Dalasjoe75!')

# ftp.cwd('html') # "test"디렉터리로 이동
# ftp.retrlines('LIST') # 디렉터리의 내용을 목록화
# #ftp.retrbinary('RETR README', open('README', 'wb').write) # README 파일 저장
# ftp.quit()


ftp.cwd('html')  # 업로드할 FTP 폴더로 이동
myfile = open(path.join(lotto_base_dir, "predict.txt"),'rb')  # 로컬 파일 열기
ftp.storbinary('STOR ' + 'predict.txt', myfile )  # 파일을 FTP로 업로드
myfile.close()  # 파일 닫기

print ("File Saved")

File Saved
